In [136]:
import math
import random

In [137]:
RHO = 4

In [138]:
def euclidGcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = euclidGcd(b % a, a)
        return (g, x - (b // a) * y, y)

def moduloInverse(a, m):
    g, x, y = euclidGcd(a, m)
    if g != 1:
        raise Exception('modulo inverse does not exist')
    else:
        return x % m

In [139]:
#Function to check if a nummber is prime
def isPrime(n):
    if n == 2:
        return True
    
    if n%2 == 0:
        return False
    
    for i in range(3, math.ceil(math.sqrt(n))+1, 2):
        if n%i == 0:
            return False
        
    return True    

In [140]:
#Function to generate a random prime number of rho bits
def generatePrime(rho=RHO):
    
    temp = random.getrandbits(rho)
    temp = temp | (1<<(rho-1)) | 1
    
    while not isPrime(temp):
        temp = random.getrandbits(rho)
        temp = temp | (1<<(rho-1)) | 1
    
    return temp

In [141]:
p = generatePrime()
q = generatePrime()

while p == q:
    q = generatePrime()
    
N = p*q

phiN = (p-1)*(q-1)

In [142]:
numAttributes = 5
pi = [None] * numAttributes
qi = [None] * numAttributes

In [ ]:
for i in range(0, numAttributes):
    temp = random.randint(2, N-1) #doubtful
    
    while math.gcd(temp, phiN) != 1:
        temp = random.randint(2, N-1)

    pi[i] = temp  
    qi[i] = moduloInverse(pi[i], phiN)

In [144]:
def checkCoprimality(k, arr):
    for i in range(len(arr)):
        if math.gcd(k, arr[i]) != 1:
            return False
    return True    

In [145]:
k = random.getrandbits(RHO)

while math.gcd(k, phiN) != 1 and checkCoprimality(k, qi) == False:
    k = random.getrandbits(RHO)

In [146]:
x = random.getrandbits(RHO)

while checkCoprimality(x, qi) == False:
    x = random.getrandbits(RHO)

In [147]:
g = random.randint(3, N-2) #doubtful
    
while math.gcd(temp, N) != 1:
    temp = random.randint(3, N-2)

In [149]:
dU = 1
for i in range(len(qi)):
    dU *= qi[i]
    